#        Holidays

In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [2]:
import pandas as pd
df = pd.read_csv("aapl_no_dates2.csv")
df.head()

,Open,High,Low,Close,Volume
0,144.88,145.30,143.10,143.50,14277848
1,143.69,144.79,142.72,144.09,21569557
2,143.02,143.50,142.41,142.73,24128782
3,142.90,144.75,142.90,144.18,19201712
4,144.11,145.95,143.37,145.06,21090636


## Using 'B' frequency is not going to help because 4th July was holiday and 'B' is not taking that into account. It only accounts for weekends

### Using CustomBusinessDay to generate US holidays calendar frequency

In [3]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [4]:
us_cal = CustomBusinessDay(calendar=USFederalHolidayCalendar())

us_cal

<CustomBusinessDay>

In [5]:
rng = pd.date_range(start="7/1/2017",end="7/23/2017", freq=us_cal)
rng
## Now 4th july won't be there

DatetimeIndex(['2017-07-03', '2017-07-05', '2017-07-06', '2017-07-07',
               '2017-07-10', '2017-07-11', '2017-07-12', '2017-07-13',
               '2017-07-14', '2017-07-17', '2017-07-18', '2017-07-19',
               '2017-07-20', '2017-07-21'],
              dtype='datetime64[ns]', freq='C')

In [6]:
df.set_index(rng, inplace=True)
df

,Open,High,Low,Close,Volume
2017-07-03,144.88,145.30,143.10,143.50,14277848
2017-07-05,143.69,144.79,142.72,144.09,21569557
2017-07-06,143.02,143.50,142.41,142.73,24128782
2017-07-07,142.90,144.75,142.90,144.18,19201712
2017-07-10,144.11,145.95,143.37,145.06,21090636
2017-07-11,144.73,145.85,144.38,145.53,19781836
2017-07-12,145.87,146.18,144.82,145.74,24884478
2017-07-13,145.50,148.49,145.44,147.77,25199373
2017-07-14,147.97,149.33,147.33,149.04,20132061
2017-07-17,148.82,150.90,148.57,149.56,23793456


#### You can define your own calendar using AbstractHolidayCalendar as shown below. USFederalHolidayCalendar is the only calendar available in pandas library and it serves as an example for those who want to write their own custom calendars. Here is the link for USFederalHolidayCalendar implementation https://github.com/pandas-dev/pandas/blob/master/pandas/tseries/holiday.py

## AbstractHolidayCalendar

In [12]:
from pandas.tseries.holiday import AbstractHolidayCalendar, nearest_workday, Holiday

class MyCalendar(AbstractHolidayCalendar):
#     """
#     US Federal Government Holiday Calendar based on rules specified by:
#     https://www.opm.gov/policy-data-oversight/
#        snow-dismissal-procedures/federal-holidays/
#     """

#     rules = [
#         Holiday("New Years Day", month=1, day=1, observance=nearest_workday),
#         USMartinLutherKingJr,
#         USPresidentsDay,
#         USMemorialDay,
#         Holiday("July 4th", month=7, day=4, observance=nearest_workday),
#         USLaborDay,
#         USColumbusDay,
#         Holiday("Veterans Day", month=11, day=11, observance=nearest_workday),
#         USThanksgivingDay,
#         Holiday("Christmas", month=12, day=25, observance=nearest_workday),
#     ]

    rules = [
        Holiday("My Birthday", month=6, day=10,observance=nearest_workday)
      ]
### observance=nearest_workday: if Holiday is in weekend then it the nearest workday is observed as Holiday

my_bday = CustomBusinessDay(calendar=MyCalendar())
rng = pd.date_range('6/1/2017','6/30/2017',freq=my_bday)   
rng

DatetimeIndex(['2017-06-01', '2017-06-02', '2017-06-05', '2017-06-06',
               '2017-06-07', '2017-06-08', '2017-06-12', '2017-06-13',
               '2017-06-14', '2017-06-15', '2017-06-16', '2017-06-19',
               '2017-06-20', '2017-06-21', '2017-06-22', '2017-06-23',
               '2017-06-26', '2017-06-27', '2017-06-28', '2017-06-29',
               '2017-06-30'],
              dtype='datetime64[ns]', freq='C')

## CustomBusinessDay

### Weekend in egypt is Friday and Saturday. Sunday is just a normal weekday and you can handle this custom week schedule using CystomBysinessDay with weekmask as shown below

In [13]:
egypt_weekdays = "Sun Mon Tue Wed Thu"

b = CustomBusinessDay(weekmask=egypt_weekdays)

pd.date_range(start="7/1/2017",periods=20,freq=b)

DatetimeIndex(['2017-07-02', '2017-07-03', '2017-07-04', '2017-07-05',
               '2017-07-06', '2017-07-09', '2017-07-10', '2017-07-11',
               '2017-07-12', '2017-07-13', '2017-07-16', '2017-07-17',
               '2017-07-18', '2017-07-19', '2017-07-20', '2017-07-23',
               '2017-07-24', '2017-07-25', '2017-07-26', '2017-07-27'],
              dtype='datetime64[ns]', freq='C')

In [14]:
b = CustomBusinessDay(holidays=['2017-07-04', '2017-07-10'], weekmask=egypt_weekdays)

pd.date_range(start="7/1/2017",periods=20,freq=b)

DatetimeIndex(['2017-07-02', '2017-07-03', '2017-07-05', '2017-07-06',
               '2017-07-09', '2017-07-11', '2017-07-12', '2017-07-13',
               '2017-07-16', '2017-07-17', '2017-07-18', '2017-07-19',
               '2017-07-20', '2017-07-23', '2017-07-24', '2017-07-25',
               '2017-07-26', '2017-07-27', '2017-07-30', '2017-07-31'],
              dtype='datetime64[ns]', freq='C')